In [78]:
# Version 3.0.1
# Title: Flow Distribution in a Doublet EGS
# Authors: Pranay Asai (UoU) & Robert Podgorney (INL)
# Edited by: Pranay Asai
# Date: 06/26/2021
# Updates:
#     1. Minor Corrections

In [79]:
#      o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|
#                     |              |              |              |
#                     /              /              /              /
#                     \              \              \              /
#                     /              /              /              /
#                     |              |              |              |
#      o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|

#                 |----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----o
#                 |              |              |              |
#                 /              /              /              /
#                 \              \              \              \
#                 /              /              /              /
#                 |              |              |              |
#  o----/\/\/\----|----/\/\/\----|----/\/\/\----|----/\/\/\----|

In [87]:
import numpy as np  #Importing NumPy4
import math
import matplotlib.pyplot as plt
import random
from ipynb.fs.defs.FrictionFactors import * #Imports all the functions

In [88]:
import platform
is_windows = any(platform.win32_ver())
if is_windows==True:
    path=r'CustomPath'
else:
    path=r'/Users/willba/Desktop/Flowratestudy'

In [89]:
#For Fracture
NumberOfFractures=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
TotalWellLength=3227 #m, Length of the well
BasePermeability=1e-12 #m2, #Fracture base Permebility
FractureHeight=100 #m Fracture entrance length
FractureWidth=5.0 #m Fracture entrance pseudo width
BaseFractureLength=100 #m Height of fractures
Area_FractureEntrance=FractureHeight*FractureWidth #Area of fracture for the fluid to enter

#For Pipe
Diameter_InjectionWell=0.18#*(0.5/0.1524)   #m Diameter of the Injection Pipe
Diameter_ProductionWell=0.18#*(0.5/0.1524)  #m Diameter of the Production Pipe
Area_InjectionWell=math.pi*Diameter_InjectionWell**2*0.25 #meters sq.
Area_ProductionWell=math.pi*Diameter_InjectionWell**2*0.25 #meters sq.
DensityWater=1000 #kg/s
e=0.015/1000  #roughness in m


#For Perforation
LengthOfPerfZone=1 #m, length og each perf zone
Cd=0.75 #Discharge Coefficient
NumberOfPerfPerMeter=1 # Number of perforations per meter
NumberOfPerfs=LengthOfPerfZone*NumberOfPerfPerMeter #Number of Perforations
BaseDiameter_InjectionPerforation=0.003175*3#*0.5 #m, Diameter of perforations
BaseDiameter_ProductionPerforation=0.003175*3#*0.5 #m, Diameter of perforations

In [90]:
#Flow Rate and Pressure

# NOTE: Change depending on which flowrate is desired to be captured
Flowrate_Initial=50 #kg/s

#storage = np.zeros((20,6))

In [91]:
#CONTROL VARIABLES

#Turn on pressure drop in injection well
InjectionWellActivate=1 #if 1, the pressure drop in the well is on
#Turn on pressure drop in production well
ProductionWellActivate=1 #if 1, the pressure drop in the well is on
#Activate Perforation
ActivateInjectionPerforations=1  #If 0, perforation pressure drop is zero.
#activate production well perforation
ActivateProductionPerforations=0

#Variable Permeablity
# If 0, All fractures have same permeability
# If 1, The permeability values are assigned at random with respect to the base value
# If 2, Option to input custom permeability values.
VariablePermeability=0

#dl = 0
#ORIENTATION OF WELLS
# If 1, The wells are Parallel
# If 2, The wells are Anti-Parallel
# If 3, The wells are Non-Parallel  # Input the Difference between first and last fracture
WellsOrientation=1


#Adaptive Perforation (Still in test phase)
# If 0, Adaptive Perforations are turned off
# If 1, Adaptive Perforations are turned on
# If 2, Custom input for Perforations
AdaptivePerf=0

# NOTE: Change the variable name to storage# with the # being the flowrate set in that run of the code
storage50 = np.zeros(len(NumberOfFractures))
print(len(NumberOfFractures))

29


In [ ]:
counter = 0
while counter <= (len(NumberOfFractures)-1):
    BaseFractureLength=100

    BaseInjectionWellSection=TotalWellLength/NumberOfFractures[counter] #m Length of Inj pipe section
    BaseProductionWellSection=TotalWellLength/NumberOfFractures[counter] #m Length of Prod pipe section



    #Initialization
    Pressure_Injection=np.zeros(NumberOfFractures[counter]+1)
    Pressure_Production=np.zeros(NumberOfFractures[counter]+1)
    Pressure_Fracture=np.zeros(NumberOfFractures[counter])
    Flowrate_Fracture=np.zeros(NumberOfFractures[counter])
    FractureLength=np.zeros(NumberOfFractures[counter])
    InjectionWellSection=np.zeros(NumberOfFractures[counter]+1)
    ProductionWellSection=np.zeros(NumberOfFractures[counter]+1)
    Permeability_Fracture=np.zeros(NumberOfFractures[counter])
    Diameter_InjectionPerforation=np.zeros(NumberOfFractures[counter])
    Diameter_ProductionPerforation=np.zeros(NumberOfFractures[counter])
    WellAngle=0
    Pressure_Injection[0]=3e7


    if WellsOrientation==3:
        #d1=float(input("Enter the difference between the first and last fracture size: "))
        #print(dl)
        d1 = 100
        BaseFractureLength=BaseFractureLength-d1/2
        WellAngle=math.atan((d1/2)/((NumberOfFractures[counter]-1)*BaseInjectionWellSection)) #Angle of Injection well
        print(math.degrees(WellAngle)*2)
    for i in range(NumberOfFractures[counter]):
        Flowrate_Fracture[i]=Flowrate_Initial/NumberOfFractures[counter]
        FractureLength[i]=abs(BaseFractureLength+(NumberOfFractures[counter]-i-1)*math.tan(WellAngle)*BaseInjectionWellSection+(NumberOfFractures[counter]-i-1)*math.tan(WellAngle)*BaseProductionWellSection)
    for i in range(NumberOfFractures[counter]+1):
        InjectionWellSection[i]=BaseInjectionWellSection/math.cos(WellAngle)
        ProductionWellSection[i]=BaseProductionWellSection/math.cos(WellAngle)


    #Calculate Fracture Permeability
    #Function is defined as:
    # var1*BasePermeability*(10^var2), where Var1 varies from 1 to 10 and Var2 varies from -1, 0 or 1, at random.
    if VariablePermeability!=2:
        Permeability_Fracture=FracturePermeability(NumberOfFractures[counter],BasePermeability,FractureWidth,FractureHeight,VariablePermeability)
    else:
        for i in range(NumberOfFractures[counter]):
            #Define any custom function
            if i%2==0:
                Permeability_Fracture[i]=BasePermeability/2
            else:
                Permeability_Fracture[i]=BasePermeability

    #Calculate Perforations
    Diameter_InjectionPerforation=AdaptivePerforation(WellsOrientation,VariablePermeability,AdaptivePerf,NumberOfFractures[counter],Diameter_InjectionPerforation,BaseDiameter_InjectionPerforation,Permeability_Fracture)
    Diameter_ProductionPerforation=AdaptivePerforation(WellsOrientation,VariablePermeability,AdaptivePerf,NumberOfFractures[counter],Diameter_ProductionPerforation,BaseDiameter_ProductionPerforation,Permeability_Fracture)
    #print(Diameter_InjectionPerforation)
    #print(Diameter_ProductionPerforation)
    #print(Area_FractureEntrance)            


    #Define a custom fucntion for Perforation diameters
    if AdaptivePerf==2:
        for i in range(NumberOfFractures[counter]-1):
            #Define any custom function
            Diameter_InjectionPerforation[i]=BaseDiameter_InjectionPerforation
            Diameter_ProductionPerforation[i]=BaseDiameter_InjectionPerforation
        Diameter_InjectionPerforation[2] = 0.0076
        Diameter_ProductionPerforation[2] = 0.0076
    #print(Diameter_InjectionPerforation)
    #print(Diameter_ProductionPerforation)



    MaxIterations=1000 #number of iterations
    #mmax=10000 #max number of iterations
    Tolerance=1e-6 #Tolerance
    FactorTest = 1000 #NumberOfFractures*FractureHeight
    Flowrate_Update=np.zeros(NumberOfFractures[counter])
    Flowrate_Frac1=np.zeros(MaxIterations)
    Flowrate_Frac2=np.zeros(MaxIterations)
    Flowrate_Frac3=np.zeros(MaxIterations)

    for j in range(MaxIterations):
        Flowrate_Cumulative=Flowrate_Initial/FactorTest
        #pressure drop in injection well
        for i in range(NumberOfFractures[counter]):
            if InjectionWellActivate==0:
                Pressure_Injection[i+1]=Pressure_Injection[i]
            else:            
                Pressure_Injection[i+1]=Pressure_Injection[i]-Fhal(e,Diameter_InjectionWell,Rep(Flowrate_Cumulative,Diameter_InjectionWell))*2*InjectionWellSection[i+1]*DensityWater*(Flowrate_Cumulative/Area_InjectionWell)**2/Diameter_InjectionWell
            Flowrate_Cumulative=Flowrate_Cumulative-Flowrate_Fracture[i]/FactorTest

        #Pressure drop in Production well
        Flowrate_Cumulative=Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest
        if WellsOrientation!=2:
            #pressure drops in last Fracture
            if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                Pressure_Production[NumberOfFractures[counter]]=Pressure_Injection[NumberOfFractures[counter]]-Pfdarcy(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,FractureLength[NumberOfFractures[counter]-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures[counter]-1])-Pperf(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,Cd,NumberOfPerfs,Diameter_InjectionPerforation[NumberOfFractures[counter]-1])-Pperf(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,Cd,NumberOfPerfs,Diameter_ProductionPerforation[NumberOfFractures[counter]-1])
            elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                Pressure_Production[NumberOfFractures[counter]]=Pressure_Injection[NumberOfFractures[counter]]-Pfdarcy(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,FractureLength[NumberOfFractures[counter]-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures[counter]-1])-Pperf(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,Cd,NumberOfPerfs,Diameter_ProductionPerforation[NumberOfFractures[counter]-1])
            elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                Pressure_Production[NumberOfFractures[counter]]=Pressure_Injection[NumberOfFractures[counter]]-Pfdarcy(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,FractureLength[NumberOfFractures[counter]-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures[counter]-1])-Pperf(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,Cd,NumberOfPerfs,Diameter_InjectionPerforation[NumberOfFractures[counter]-1])
            elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations
                Pressure_Production[NumberOfFractures[counter]]=Pressure_Injection[NumberOfFractures[counter]]-Pfdarcy(Flowrate_Fracture[NumberOfFractures[counter]-1]/FactorTest,FractureLength[NumberOfFractures[counter]-1],Area_FractureEntrance,Permeability_Fracture[NumberOfFractures[counter]-1])


            for i in range(NumberOfFractures[counter],0,-1):
                #turning off production well pressure drop
                if ProductionWellActivate==0:
                    Pressure_Production[i-1]=Pressure_Production[i]
                else:
                    Pressure_Production[i-1]=Pressure_Production[i]-Fhal(e,Diameter_ProductionWell,Rep(Flowrate_Cumulative,Diameter_ProductionWell))*2*ProductionWellSection[i-1]*DensityWater*(Flowrate_Cumulative/Area_ProductionWell)**2/Diameter_ProductionWell
                Flowrate_Cumulative=Flowrate_Cumulative+Flowrate_Fracture[i-1]/FactorTest
            #Flow Rate in all Fractures
            Flowrate_Reinitilized=0
            for i in range(NumberOfFractures[counter]):
                Pressure_Fracture[i]=Pressure_Injection[i+1]-Pressure_Production[i+1]
                if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                    Flowrate_Fracture[i]=Q_all_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_InjectionPerforation[i],Diameter_ProductionPerforation[i])*FactorTest
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                    Flowrate_Fracture[i]=Q_Production_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_ProductionPerforation[i])*FactorTest
                elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                    Flowrate_Fracture[i]=Q_Injection_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_InjectionPerforation[i])*FactorTest
                elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations            
                    Flowrate_Fracture[i]=Q_No_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i])*FactorTest

                Flowrate_Reinitilized=Flowrate_Reinitilized+Flowrate_Fracture[i]
        else:
            #pressure drops in First Fracture
            if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs,Diameter_InjectionPerforation[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs,Diameter_ProductionPerforation[0])
            elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs,Diameter_ProductionPerforation[0])
            elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations    
                Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])-Pperf(Flowrate_Fracture[0]/FactorTest,Cd,NumberOfPerfs,Diameter_InjectionPerforation[0])
            elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations    
                Pressure_Production[0]=Pressure_Injection[1]-Pfdarcy(Flowrate_Fracture[0]/FactorTest,FractureLength[0],Area_FractureEntrance,Permeability_Fracture[0])

            #Pressure drop in Production well
            Flowrate_ProductionReinitilized=0
            for i in range(NumberOfFractures[counter]):
                Flowrate_ProductionReinitilized=Flowrate_ProductionReinitilized+Flowrate_Fracture[i]/FactorTest
                if ProductionWellActivate==0:
                    Pressure_Production[i+1]=Pressure_Production[i]
                else:
                    Pressure_Production[i+1]=Pressure_Production[i]-Fhal(e,Diameter_ProductionWell,Rep(Flowrate_ProductionReinitilized,Diameter_ProductionWell))*2*ProductionWellSection[i-1]*DensityWater*(Flowrate_ProductionReinitilized/Area_ProductionWell)**2/Diameter_ProductionWell

            #Flow Rate in all Fractures including perf
            Flowrate_Reinitilized=0
            for i in range(NumberOfFractures[counter]):
                Pressure_Fracture[i]=Pressure_Injection[i+1]-Pressure_Production[i]
                if Pressure_Fracture[i]>0:
                    if ActivateInjectionPerforations==1 and ActivateProductionPerforations==1: #Activate all perforations
                        Flowrate_Fracture[i]=Q_all_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_InjectionPerforation[i],Diameter_ProductionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==1: #Activate only production perforations
                        Flowrate_Fracture[i]=Q_Production_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_ProductionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==1 and ActivateProductionPerforations==0: #Activate only injection perforations
                        Flowrate_Fracture[i]=Q_Injection_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i],Cd,NumberOfPerfs,Diameter_InjectionPerforation[i])*FactorTest
                    elif ActivateInjectionPerforations==0 and ActivateProductionPerforations==0: #Turn off all perforations            
                        Flowrate_Fracture[i]=Q_No_perfs(Pressure_Fracture[i],FractureLength[i],Area_FractureEntrance,Permeability_Fracture[i])*FactorTest                

                else:
                    Flowrate_Fracture[i]=0.01
                Flowrate_Reinitilized=Flowrate_Reinitilized+Flowrate_Fracture[i]

        
        #Calculating flowrate for next iteration
        for i in range(NumberOfFractures[counter]):
            Flowrate_Fracture[i]=(Flowrate_Initial)*Flowrate_Fracture[i]/Flowrate_Reinitilized

        Flowrate_Update=Flowrate_Update+np.array(Flowrate_Fracture)
        if j>50:
            Flowrate_Fracture=Flowrate_Update/j
    print(counter)
    
    distpct=np.empty(NumberOfFractures[counter])
    for i in range(NumberOfFractures[counter]):
        distpct[i]=100*(Flowrate_Fracture[i]/sum(Flowrate_Fracture))
    print(distpct)
    ppp = (145.038*(max(Pressure_Injection)-min(Pressure_Production))/1e6)
    
    
    # NOTE: Change the variable name to storage# with the # being the flowrate set in that run of the code
    storage50[counter] = ppp
    print(ppp)
    counter = counter + 1
print(Flowrate_Initial)

In [93]:

## NOTE: Must change the flowrate everytime to get a new value
# ALSO: change the name from storage5 to storage25 depending on the flowrate value

print(storage50)
np.savetxt("FRACTUREstorage50.csv", storage50, delimiter=",", header = ','.join(['Pressure Drop']))

[16836.23299345  7745.12148225  4523.39506086  3013.90346038
  2183.97416368  1677.49624058  1344.78622034  1113.89625866
   946.70272866   821.44742006   724.96334907   648.89560497
   587.73077193   537.70862994   496.19156329   461.2839887
   431.59448909   406.08254652   383.95711226   364.60787646
   347.55770419   332.42909155   318.92010293   306.78684076
   295.83049355   285.88764259   276.82292134   268.52339571
   260.89421834]
